In [35]:
import numpy as np
import pandas as pd
from slither_sol_helpers import *
import os, json
from tqdm import tqdm
import swifter
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)


from swifter import set_defaults
set_defaults(
    npartitions=2*os.cpu_count(),
    dask_threshold=1,
    scheduler="processes",
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=False,
    force_parallel=False,
)

partial_dataset_path = '/home/pippertetsing/sourcify_contract_data/partial_match'
full_dataset_path = '/home/pippertetsing/sourcify_contract_data/full_match'
contracts_dirs_saved = './contracts_dirs.pkl'

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# read all contract dirs or load it if already done
# contracts_dirs_partial = check_folder_structure(partial_dataset_path)
# contracts_dirs_full = check_folder_structure(full_dataset_path)
# contracts_dirs = pd.concat([contracts_dirs_full, contracts_dirs_partial])

contracts_dirs = pd.read_pickle(contracts_dirs_saved)

In [10]:
# Filter and retain only dirs wit '''sources''' dirctories
contracts_dirs = contracts_dirs[contracts_dirs.has_src_files == True]
contracts_dirs

,contracts_dir,has_src_files
0,/home/pippertetsing/sourcify_contract_data/ful...,True
1,/home/pippertetsing/sourcify_contract_data/ful...,True
2,/home/pippertetsing/sourcify_contract_data/ful...,True
3,/home/pippertetsing/sourcify_contract_data/ful...,True
4,/home/pippertetsing/sourcify_contract_data/ful...,True
...,...,...
382057,/home/pippertetsing/sourcify_contract_data/par...,True
382058,/home/pippertetsing/sourcify_contract_data/par...,True
382059,/home/pippertetsing/sourcify_contract_data/par...,True
382060,/home/pippertetsing/sourcify_contract_data/par...,True


In [4]:
sub_set = contracts_dirs.sample(10)
sub_set.iloc[0].contracts_dir

'/home/pippertetsing/sourcify_contract_data/full_match/5/0x417a9dAb55aD76aDb1D3AD9741942Bc23b288Afb'

In [8]:
# Small test 
small_set = []
for idx in tqdm(range(len(sub_set))):
    row = sub_set.iloc[idx]
    src_dir = row.contracts_dir + '/sources'
    args = construct_mapping_and_args(row.contracts_dir + "/metadata.json", True, True)

    if get_all_sol_files(src_dir)is None:
        print(f'no solidity file in {src_dir}')
    else:
        for sol_file_cp in get_all_sol_files(src_dir):
            sol_file = sol_file_cp.replace(src_dir, '.')
            cmd = ['slither', sol_file.replace(src_dir, '.')]
            _ = [cmd.append(x) for x in args]
            p = subprocess.run(cmd,
                cwd=src_dir,
                shell=False,                            
                capture_output = True,
                universal_newlines=True)
            
            if p.stdout == '':
                small_set.append({'source_dir':src_dir, 'sol_file':sol_file_cp,
                               'processed':False,
                                'slither':None})
                #print(sol_file, 'process status:', False)
            else:
                output = json.loads(p.stdout)
                #print(sol_file, 'process status:', output['success'])
                small_set.append({'source_dir':src_dir, 'sol_file':sol_file_cp,
                               'processed':output['success'],
                                'slither':get_slither_check_from_json(output)})
                        

100%|██████████| 10/10 [00:35<00:00,  3.56s/it]


In [36]:
def slither_process(df_row):
    
    result = []
    src_dir = df_row.contracts_dir + '/sources'
    args = construct_mapping_and_args(df_row.contracts_dir + "/metadata.json", True, True)

    if get_all_sol_files(src_dir) is None:
        print(f'no solidity file in {src_dir}')
    else:
        for sol_file_cp in get_all_sol_files(src_dir):
            sol_file = sol_file_cp.replace(src_dir, '.')
            cmd = ['slither', sol_file.replace(src_dir, '.')]
            _ = [cmd.append(x) for x in args]
            p = subprocess.run(cmd,
                cwd=src_dir,
                shell=False,                            
                capture_output = True,
                universal_newlines=True)
            
            if p.stdout == '':
                result.append({'source_dir':src_dir, 'sol_file':sol_file_cp,
                                'contracts_dirs':df_row.contracts_dir,
                                'has_src_files': df_row.has_src_files,
                               'slither_processed':False,
                                'slither':None})
                #print(sol_file, 'process status:', False)
            else:
                output = json.loads(p.stdout)
                #print(sol_file, 'process status:', output['success'])
                result.append({'source_dir':src_dir, 'sol_file':sol_file_cp,
                               'slither_processed':output['success'],
                               'contracts_dirs':df_row.contracts_dir,
                               'has_src_files': df_row.has_src_files,
                               'slither':get_slither_check_from_json(output)})
    return pd.DataFrame(result)

sub_set['slither_res'] = sub_set.parallel_apply(slither_process, axis=1) 

ModuleNotFoundError: No module named 'ipywidgets'

In [24]:
dataset = pd.DataFrame()

for _, row in sub_set.iterrows():
    dataset = pd.concat([dataset, row.slither_res])

In [25]:
dataset

,source_dir,sol_file,processed,contracts_dirs,has_src_files,slither
0,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None
1,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None
2,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,"[23, 23, 23, 23, 23, 23]"
3,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None
4,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None
5,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None
6,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None
0,/home/pippertetsing/sourcify_contract_data/par...,/home/pippertetsing/sourcify_contract_data/par...,True,/home/pippertetsing/sourcify_contract_data/par...,True,"[84, 84, 84, 84, 84, 84, 84, 80, 80, 77, 77, 7..."
0,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None
1,/home/pippertetsing/sourcify_contract_data/ful...,/home/pippertetsing/sourcify_contract_data/ful...,True,/home/pippertetsing/sourcify_contract_data/ful...,True,None


In [15]:
sub_set

,contracts_dir,has_src_files,slither_res
143528,/home/pippertetsing/sourcify_contract_data/ful...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
175078,/home/pippertetsing/sourcify_contract_data/par...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
540614,/home/pippertetsing/sourcify_contract_data/ful...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
95290,/home/pippertetsing/sourcify_contract_data/ful...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
6905,/home/pippertetsing/sourcify_contract_data/par...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
539117,/home/pippertetsing/sourcify_contract_data/ful...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
117864,/home/pippertetsing/sourcify_contract_data/par...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
513973,/home/pippertetsing/sourcify_contract_data/ful...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
446998,/home/pippertetsing/sourcify_contract_data/ful...,True,[{'source_dir': '/home/pippertetsing/sourcify_...
49195,/home/pippertetsing/sourcify_contract_data/ful...,True,[{'source_dir': '/home/pippertetsing/sourcify_...


In [ ]:
r = './full_match/11155111/0xbdCB29d71eBCDb7114Cfb8c6d9298B4976D6a608'
arg = construct_mapping_and_args(r + "/metadata.json")
#os.chdir(r + '/sources')

cmd = ['slither', '_openzeppelin/contracts/proxy/Proxy.sol']
for x in arg:
    cmd.append(x)
print(cmd)

p = subprocess.run(cmd,
                cwd=r+'/sources',
                shell=False,                            
                capture_output = True,
                universal_newlines=True)
# p = subprocess.run(['slither', '_openzeppelin/contracts/proxy/Proxy.sol', '--solc-args', '--optimize --optimize-runs 200',
#                         '--solc-solcs-select', '0.8.9', '--exclude-informational', '--exclude-dependencies', '--exclude-optimization'],
#                         cwd=r+'/sources',
#                         shell=False,
#                         capture_output = True,
#                         universal_newlines=True
#                         )
# os.system("slither _openzeppelin/contracts/proxy/Proxy.sol " + arg)

In [ ]:
out = json.loads(p.stdout)

In [ ]:
detectors.get('abiencoderv2-array').get('idx')

In [9]:
os.cpu_count()

8